## Imports

In [2]:
import pyodbc
import pandas as pd
import numpy as np
import os

In [80]:
year = 2021

if year == 2021:
    conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
            r'DBQ=C:\Users\mit\Documents\wildfires\data\sit\databases' + '\\' f'{year}_SIT209_DATA.accdb;')
elif year > 2014:
    conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
            r'DBQ=C:\Users\mit\Documents\wildfires\data\sit\databases' + '\\' f'{year} SIT DATA.accdb;')
else:
    conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
            r'DBQ=C:\Users\mit\Documents\wildfires\data\sit\databases' + '\\' f'sit_{year}.accdb;')

In [82]:
conn = pyodbc.connect(conn_str)

if year == 2021:
    
    incidents_df = pd.read_sql('select * from 19_SIT209_HISTORY_INCIDENTS', conn)
    reports_df = pd.read_sql('select * from 18_SIT209_HISTORY_INCIDENT_209_REPORTS', conn)

    reports_df['INC209R_IDENTIFIER'] = reports_df['INC209R_IDENTIFIER'].astype(int).astype(str)

    try:
        lookup = pd.read_sql('select * from 8_SIT209_HISTORY_SIT209_LOOKUP_CODES', conn)
    except:
        lookup = pd.read_sql('select * from SIT209_LOOKUP_CODES', conn)

    inctyp_lookup = lookup[lookup['CODE_TYPE'] == 'INCIDENT_TYPE']
    inctyp_lookup = dict(zip(inctyp_lookup['LUCODES_IDENTIFIER'], inctyp_lookup['CODE_NAME'])) 
    incidents_df['INCTYP_IDENTIFIER'] = incidents_df['INCTYP_IDENTIFIER'].astype(str).map(inctyp_lookup)

    resources_lookup = lookup[lookup['CODE_TYPE'] == 'RESOURCE_TYPE']
    resources_lookup = dict(zip(resources_lookup['LUCODES_IDENTIFIER'], resources_lookup['CODE_NAME']))

    resources_df = pd.read_sql('select * from 26_SIT209_HISTORY_INCIDENT_209_RES_UTILIZATIONS', conn)

    resources_df['INC209R_IDENTIFIER'] = resources_df['INC209R_IDENTIFIER'].astype(int).astype(str)

    resources_df = resources_df[resources_df['INC209R_IDENTIFIER'].isin(reports_df['INC209R_IDENTIFIER'])]
    resources_df['resource'] = resources_df['RESTYP_IDENTIFIER'].map(resources_lookup)

else:
    incidents_df = pd.read_sql('select * from SIT209_HISTORY_INCIDENTS', conn)
    reports_df = pd.read_sql('select * from SIT209_HISTORY_INCIDENT_209_REPORTS', conn)

    reports_df['INC209R_IDENTIFIER'] = reports_df['INC209R_IDENTIFIER'].astype(int).astype(str)

    try:
        lookup = pd.read_sql('select * from SIT209_HISTORY_SIT209_LOOKUP_CODES', conn)
    except:
        lookup = pd.read_sql('select * from SIT209_LOOKUP_CODES', conn)

    inctyp_lookup = lookup[lookup['CODE_TYPE'] == 'INCIDENT_TYPE']
    inctyp_lookup = dict(zip(inctyp_lookup['LUCODES_IDENTIFIER'], inctyp_lookup['CODE_NAME'])) 
    incidents_df['INCTYP_IDENTIFIER'] = incidents_df['INCTYP_IDENTIFIER'].astype(str).map(inctyp_lookup)

    resources_lookup = lookup[lookup['CODE_TYPE'] == 'RESOURCE_TYPE']
    resources_lookup = dict(zip(resources_lookup['LUCODES_IDENTIFIER'], resources_lookup['CODE_NAME']))

    resources_df = pd.read_sql('select * from SIT209_HISTORY_INCIDENT_209_RES_UTILIZATIONS', conn)

    resources_df['INC209R_IDENTIFIER'] = resources_df['INC209R_IDENTIFIER'].astype(int).astype(str)

    resources_df = resources_df[resources_df['INC209R_IDENTIFIER'].isin(reports_df['INC209R_IDENTIFIER'])]
    resources_df['resource'] = resources_df['RESTYP_IDENTIFIER'].map(resources_lookup)

DatabaseError: Execution failed on sql 'select * from 18_SIT209_HISTORY_INCIDENT_209_REPORTS': ('HY024', "[HY024] [Microsoft][ODBC Microsoft Access Driver] 'C:\\Excel\\Annual_reports\\Access_db\\2021_cy_csv_files\\18_2_SIT209_HISTORY_INCIDENT_209_REPORTS.xlsx' is not a valid path. Make sure that the path name is spelled correctly and that you are connected to the server on which the file resides. (-1023) (SQLExecDirectW)")

In [74]:
resources_df['RESOURCE_QUANTITY'] = resources_df['RESOURCE_QUANTITY'].apply(lambda x: x if x != '' else 0)
resources_df['RESOURCE_QUANTITY'] = resources_df['RESOURCE_QUANTITY'].astype(float)

In [75]:
resources_df = resources_df[['INC209RU_IDENTIFIER', 'INC209R_IDENTIFIER', 'NWCGAG_IDENTIFIER', 'RESTYP_IDENTIFIER', 'resource', 'RESOURCE_QUANTITY', 'RESOURCE_PERSONNEL']]

In [76]:
resources_df[resources_df['resource'] == 'Crew, Type 1'].sort_values('RESOURCE_QUANTITY', ascending=False).head(50)

,INC209RU_IDENTIFIER,INC209R_IDENTIFIER,NWCGAG_IDENTIFIER,RESTYP_IDENTIFIER,resource,RESOURCE_QUANTITY,RESOURCE_PERSONNEL
47118,760617.0,760591,22406.0,9908.0,"Crew, Type 1",123.0,240.0
29098,763125.0,763101,22406.0,9908.0,"Crew, Type 1",123.0,240.0
60273,1030711.0,1030679,1000120.0,9908.0,"Crew, Type 1",61.0,1015.0
64955,1035341.0,1035310,1000120.0,9908.0,"Crew, Type 1",61.0,1014.0
62227,1041148.0,1041117,1000120.0,9908.0,"Crew, Type 1",61.0,1019.0
62162,1040622.0,1040591,1000120.0,9908.0,"Crew, Type 1",61.0,1017.0
61584,1035550.0,1035519,1000120.0,9908.0,"Crew, Type 1",61.0,1014.0
64508,1026508.0,1026474,1000120.0,9908.0,"Crew, Type 1",61.0,1013.0
61618,1040985.0,1040954,1000120.0,9908.0,"Crew, Type 1",61.0,1017.0
64429,1025641.0,1025606,1000120.0,9908.0,"Crew, Type 1",61.0,1012.0


In [77]:
cols = \
['INC209R_IDENTIFIER',
 'INC_IDENTIFIER',
 'REPORT_FROM_DATE',
 'REPORT_TO_DATE',
 'STATUS',
 'CURR_INCIDENT_AREA',
 'PCT_CONTAINED_COMPLETED',
 'COMPLEXITY_LEVEL_IDENTIFIER',
 'FUEL_MODEL_IDENTIFIER',
 'PROJ_INCIDENT_AREA',
 'PROJ_INC_AREA_UOM_IDENTIFIER',
 'ANTICIPATED_COMPLETION_DATE',
 'EXPECTED_CONTAINMENT_DATE',
 'PCT_PERIM_TO_BE_CONTAINED',
 'SECNDRY_FUEL_MODEL_IDENTIFIER',
 'ADDNTL_FUEL_MODEL_IDENTIFIER',
 'GEN_FIRE_BEHAVIOR_IDENTIFIER',
 'FIRE_BEHAVIOR_1_IDENTIFIER',
 'FIRE_BEHAVIOR_2_IDENTIFIER',
 'FIRE_BEHAVIOR_3_IDENTIFIER']

In [79]:
reports_df

,INC209R_IDENTIFIER,INC_IDENTIFIER,DONWCGU_PROT_UNIT_IDENTIFIER,REPORT_FROM_DATE,REPORT_TO_DATE,STATUS,INCIDENT_DESCRIPTION,INCIDENT_COMMANDERS_NARR,CURR_INCIDENT_AREA,CURR_INC_AREA_UOM_IDENTIFIER,...,PCT_PERIM_TO_BE_CONTAINED,GACC_PRIORITY,DISPATCH_PRIORITY,STRATEGIC_OBJECTIVES,POO_LD_PM_IDENTIFIER,ADDTNL_COOP_ASSIST_ORG_NARR,SINGLE_COMPLEX_FLAG,INCIDENT_NUMBER,INCIDENT_NAME,LOCAL_TIMEZONE_IDENTIFIER
0,289621,289620.0,9021.0,2014-02-27 02:00:00,2014-02-27 22:45:00,I,None,Joseph Debree,400.0,20928.0,...,0.0,NaN,NaN,null,NaN,null,S,2014-16-0128,Brahma,NaN
1,290867,290866.0,9684.0,2014-02-28 14:00:00,2014-02-28 20:00:00,I,None,Charles Tice,NaN,NaN,...,100.0,NaN,NaN,None,NaN,None,S,014002,173,NaN
2,290914,290913.0,9727.0,2014-02-28 15:30:00,2014-03-01 07:00:00,I,None,Stanley,200.0,20928.0,...,100.0,NaN,NaN,Full containment,NaN,None,S,14006023,Weller Yard,NaN
3,247521,247520.0,9211.0,2014-01-12 11:45:00,2014-01-12 18:30:00,F,None,Erin Jones\r\n\r\nUnified Command,1300.0,20928.0,...,100.0,NaN,NaN,None,NaN,None,S,42025,Post Road Fire,NaN
4,247533,247532.0,9211.0,2014-01-12 14:00:00,2014-01-12 18:00:00,F,None,Michael Montgomery\n\nUnified Command,400.0,20928.0,...,100.0,NaN,NaN,None,NaN,None,S,42026,College Ave. Fire,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6180,1080354,710365.0,246749.0,2014-10-02 19:00:00,2014-10-09 19:00:00,U,None,"ICT4 Matthew H. Young DFMO Powell, Lochsa/Powe...",10.0,20928.0,...,0.0,NaN,NaN,Allow fire to accomplish resource benefit obje...,20852.0,None.,S,000298,Big Sand,NaN
6181,1080381,816006.0,246749.0,2014-10-02 19:00:00,2014-10-09 19:00:00,U,None,"ICT4, Matthew H. Young, Powell DFMO, Lochsa/Po...",15.0,20928.0,...,0.0,NaN,NaN,Allow the fire to accomplish resource benefit ...,20852.0,None.,S,000522,White Sand,NaN
6182,1074694,983669.0,8558.0,2014-10-07 08:00:00,2014-10-07 20:00:00,U,None,Jerry McGowan (CIIMT-1)\n,97717.0,20928.0,...,100.0,NaN,NaN,Full containment of the fire.\n\n Completion ...,NaN,"CAL-OES, California Conservation Corps, CalTr...",S,023461,KING,NaN
6183,1075675,1069077.0,484042.0,2014-10-07 10:00:00,2014-10-07 22:00:00,F,None,"Mitch Williams, Oregon Dept. of Forestry",260.0,20928.0,...,100.0,NaN,NaN,achieve 100% containment by end of shift tuesday,NaN,null,S,001369,Mt. Harris 14,NaN


In [78]:
reports_df[cols]

KeyError: "['SECNDRY_FUEL_MODEL_IDENTIFIER', 'ADDNTL_FUEL_MODEL_IDENTIFIER', 'GEN_FIRE_BEHAVIOR_IDENTIFIER', 'FIRE_BEHAVIOR_1_IDENTIFIER', 'FIRE_BEHAVIOR_2_IDENTIFIER', 'FIRE_BEHAVIOR_3_IDENTIFIER'] not in index"

In [67]:
out_path = r'C:\Users\mit\Dropbox (MIT)\wildfire_repo\sit_209\20220624'
if not os.path.isdir(out_path):
    os.mkdir(out_path)

In [68]:
save_path = out_path + f'\\{year}'

if not os.path.isdir(save_path):
    os.mkdir(save_path)
    
incidents_df.to_csv(f'{save_path}/incidents.csv', index=False)
reports_df[cols].to_csv(f'{save_path}/reports_small.csv', index=False)
reports_df.to_csv(f'{save_path}/reports_full.csv', index=False)
resources_df.to_csv(f'{save_path}/resources.csv', index=False)
lookup.to_csv(f'{save_path}/column_lookup.csv', index=False)

In [3]:
stden_path = r"C:\Users\mit\Downloads\Data for Jacob May 4 2022\data\raw\ics-209-plus-2.0-pre-release-v1\ics-209-plus-2.0-pre-release-v1\ics209-plus_sitreps_1999to2020.csv"
cols = ['INC_IDENTIFIER', 'INC209R_IDENTIFIER', 'REPORT_FROM_DATE', 'REPORT_TO_DATE']
stden = pd.read_csv(stden_path,  usecols=cols)
stden = stden[stden['REPORT_FROM_DATE'].notna()]
stden['REPORT_FROM_DATE'] = pd.to_datetime(stden['REPORT_FROM_DATE'])
stden['REPORT_TO_DATE'] = pd.to_datetime(stden['REPORT_TO_DATE'])
stden['year'] = stden['REPORT_FROM_DATE'].apply(lambda x: x.year)
stden = stden[stden['year'] >= 2014]
stden['year'].value_counts()
for col in ['INC209R_IDENTIFIER', 'INC_IDENTIFIER']:
    stden[col] = stden[col].astype(int).astype(str)
    
stden.to_csv(f"{out_path}/report_id_lookup.csv", index=False)

C:\Users\mit\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (103) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
out_path = r'C:\Users\mit\Dropbox (MIT)\wildfire_repo\sit_209'
stden_path = r"C:\Users\mit\Downloads\Data for Jacob May 4 2022\data\raw\ics-209-plus-2.0-pre-release-v1\ics-209-plus-2.0-pre-release-v1\ics209-plus_sitreps_1999to2020.csv"
stden = pd.read_csv(stden_path)
stden = stden[stden['REPORT_FROM_DATE'].notna()]
stden['REPORT_FROM_DATE'] = pd.to_datetime(stden['REPORT_FROM_DATE'])
stden['REPORT_TO_DATE'] = pd.to_datetime(stden['REPORT_TO_DATE'])
for col in ['INC209R_IDENTIFIER', 'INC_IDENTIFIER']:
    stden[col] = stden[col].astype(int).astype(str)
    
stden.to_csv(f"{out_path}/clean_reports.csv", index=False)

In [117]:
stden.isna().sum()

INC209R_IDENTIFIER    0
INC_IDENTIFIER        0
REPORT_FROM_DATE      0
REPORT_TO_DATE        0
year                  0
dtype: int64

In [297]:
year = 2018

In [298]:
join = r"C:\Users\mit\Downloads\Data for Jacob May 4 2022\ICS-209 to ROSS and IROC join\ICS-209 to ROSS and IROC join\Autojoin_step4_"
join = f"{join}{year}.csv"
inc_path = out_path + f'\\{year}' + '/incidents.csv'
assignment_path = r"C:\Users\mit\Downloads\Data for Jacob May 4 2022\data\processed" + f'\\{year}\\true_assignments.csv' 

In [299]:
incs = pd.read_csv(inc_path)
join_df = pd.read_csv(join)

incs['INCIDENT_IDENTIFIER'] = incs['INCIDENT_IDENTIFIER'].astype(int)
join_df['Inc ID'] = pd.to_numeric(join_df['Inc ID'], errors='coerce')
join_df = join_df[join_df['Inc ID'].notna()]
try:
    lookup = dict(zip(join_df['Inc ID'].astype(int), join_df['INC_IDENTIFIER']))
except:
    lookup = dict(zip(join_df['Inc ID'].astype(int), join_df['INC_IDENTIFER']))

In [300]:
assignments = pd.read_csv(assignment_path)
assignments['INCIDENT_IDENTIFIER'] = assignments['Inc ID'].map(lookup)

In [301]:
assignments['day_mob'] = pd.to_datetime(assignments['Mob Start']).apply(lambda x: x.date())
assignments['day_demob'] = pd.to_datetime(assignments['Demob Start']).apply(lambda x: x.date())

In [302]:
with_inc = assignments[assignments['INCIDENT_IDENTIFIER'].notna()]
mobs = with_inc.groupby(['INCIDENT_IDENTIFIER', 'day_mob'], as_index=False).agg({'uuid' : 'count'})
demobs = with_inc.groupby(['INCIDENT_IDENTIFIER', 'day_demob'], as_index=False).agg({'uuid' : 'count'})
merged = pd.merge(mobs, demobs, how='outer', left_on=['INCIDENT_IDENTIFIER', 'day_mob'],
                 right_on=['INCIDENT_IDENTIFIER', 'day_demob'], suffixes = ['_mob', '_demob'])
merged['day'] = merged['day_mob'].combine_first(merged['day_demob'])
cols = ['day', 'INCIDENT_IDENTIFIER', 'uuid_mob', 'uuid_demob']
new_cols = ['day', 'INCIDENT_IDENTIFIER', 'crews_dispatched', 'crews_demobilized']
merged = merged[cols].fillna(0).rename(columns=dict(zip(cols, new_cols)))
merged = merged.sort_values(['day', 'INCIDENT_IDENTIFIER']).reset_index(drop=True)

In [303]:
assignments['Inc ID'].map(lookup).notna().value_counts()

True     1135
False     653
Name: Inc ID, dtype: int64

In [304]:
lookup_path = r'C:\Users\mit\Dropbox (MIT)\wildfire_repo\sit_209\20220624\crew_info'
if not os.path.isdir(lookup_path):
    os.mkdir(lookup_path)
merged.to_csv(f'{lookup_path}/{year}.csv', index=False)

In [314]:
incs.sort_values('INCIDENT_NAME').iloc[5:16]

,ID,INCIDENT_IDENTIFIER,INCIDENT_NUMBER,DONWCGU_PROT_UNIT_IDENTIFIER,INCIDENT_NAME,INCTYP_IDENTIFIER,CAUSE_IDENTIFIER,DISCOVERY_DATE,POO_LATITUDE,POO_LONGITUDE,...,EXT_SYSTEM_LAST_MODIFIED_DATE,FIRECODE,NWCG_PROT_UNIT_IDENTIFIER,LOCAL_TIMEZONE_IDENTIFIER,CY,COMPLEXITY_LEVEL_NARR,INCIDENT_COMMANDERS_NARR,COMPLEXITY_LEVEL_IDENTIFIER,UNIFIED_COMMAND_FLAG,INC_MGMT_ORG_IDENTIFIER
21555,21556,9196255,001373,NaN,Alamosa,NaN,7483986.0,2018-07-05 19:24:09,37.37634,-106.34560,...,2018-07-20 12:45:34,NaN,21038883.0,NaN,2018.0,NaN,Glines,NaN,N,NaN
23240,23241,9263938,030613,7470335.0,Arc,NaN,7483985.0,2018-09-08 15:26:00,39.55181,-119.52030,...,2018-10-21 15:43:11,L4SN,21038250.0,7484142.0,2018.0,NaN,BC 70 Shane Dickson,7483838.0,N,7484005.0
12909,12910,9126923,000018,NaN,Ashley Refuse,NaN,7483985.0,2018-04-21 14:27:00,48.10172,-114.56850,...,2018-05-01 16:14:44,NaN,21038602.0,NaN,2018.0,NaN,Marx,NaN,N,NaN
18255,18256,9225327,180057,NaN,Astor,NaN,7483988.0,2018-04-22 15:39:27,44.03789,-85.86707,...,2018-04-27 11:42:20,NaN,21038011.0,NaN,2018.0,NaN,Nichols,NaN,N,NaN
13602,13603,9102865,000716,NaN,BABES,NaN,7483986.0,2018-07-18 09:32:12,35.07129,-111.97500,...,2018-08-03 13:15:55,NaN,21041549.0,NaN,2018.0,NaN,NaN,NaN,N,NaN
11649,11650,9102867,001233,NaN,BERRY,NaN,7483988.0,2018-07-18 20:11:00,35.15929,-113.69720,...,2018-07-19 14:47:11,L0YB,21041718.0,NaN,2018.0,NaN,NaN,NaN,N,NaN
11821,11822,9115533,001748,NaN,BIG MEADOW,NaN,7483988.0,2018-08-04 14:52:00,38.41600,-120.10970,...,2018-08-15 12:24:19,L2MV,21040448.0,NaN,2018.0,NaN,Capt 613,NaN,N,NaN
6621,6622,9085974,000962,NaN,BLACK TOWER,NaN,7483985.0,2018-10-09 11:29:00,31.60358,-110.44170,...,2018-12-14 15:20:13,L5S0,21041421.0,NaN,2018.0,NaN,TAVARES,NaN,N,NaN
11863,11864,9121926,000039,NaN,BUGGELN,NaN,7483988.0,2018-05-28 21:48:00,35.97946,-111.95670,...,2018-05-31 17:40:53,LT0X,21041592.0,NaN,2018.0,NaN,HEFRICH,NaN,N,NaN
18068,18069,9218867,001645,NaN,Badger Creek,NaN,7483986.0,2018-07-22 11:15:00,38.69856,-105.78000,...,2018-07-22 17:04:17,L06Y,21039732.0,NaN,2018.0,NaN,Tallahesse FD,NaN,N,NaN


In [313]:
assignments.sort_values('Inc Name').iloc[300:350]

,uuid,Res ID,Res Name,Filled Cat Item Name,Inc ID,Inc Name,Inc Number,Inc Type,Inc Lat,Inc Long,...,Demob End,Demob End TZ,demob_start_norm,demob_end_norm,inc_gacc,rest_after,assoc_prepo,INCIDENT_IDENTIFIER,day_mob,day_demob
1099,b1cc13b6-2fc1-4acc-a4ed-8093da69aa26,58471,CREW - T2IA - Kern Valley,"Crew, Type 2 IA",285442,ANCHORITE,NV-HTF-001198,Fire - Wildfire,38.178888,-118.726388,...,2018-07-24 14:30:00,PST,2018-07-24 08:00:00,2018-07-24 14:30:00,GBC,True,aa11ae8a-bb06-4009-9211-d87827e33de3,NaN,2018-07-18,2018-07-24
116,c48a7c36-82a0-4a46-9264-fe9d95dea7a2,1340,NaN,"Crew, Type 1",277976,ANCIENT LAKES,WA-SPD-000140,Fire - Wildfire,47.155277,-119.974721,...,2018-06-08 18:30:00,PST,2018-06-08 17:00:00,2018-06-08 18:30:00,NWC,True,not_associated,NaN,2018-06-08,2018-06-08
1114,f9403da2-8171-415c-86d1-140045f6f626,60740,CREW - T2IA - NMCIF - MT. TAYLOR,"Crew, Type 2 IA",274495,ANDREWS RX,NM-CIF-000082,Fire - Prescribed,35.190277,-108.159166,...,2018-04-07 19:00:00,MST,2018-04-07 16:00:00,2018-04-07 18:00:00,SWC,False,not_associated,NaN,2018-04-05,2018-04-07
897,339bd089-0a60-43f3-a81c-5d6d57bf5626,54657,CREW - T1 - Texas Canyon Hotshots IHC,"Crew, Type 1",307197,ANTONIO,CA-ANF-004246,Fire - Wildfire,34.288610,-117.668332,...,2018-11-12 17:30:00,PST,2018-11-12 16:00:00,2018-11-12 17:30:00,OSCC,True,not_associated,NaN,2018-11-11,2018-11-12
637,b63c32c8-6240-4e78-a96c-a3ed7fdcd693,31280,NaN,"Crew, Type 1",280853,APPLEVUN,AK-UYD-000305,Fire - Wildfire,66.633054,-144.098888,...,2018-07-15 18:10:00,AST,2018-07-15 15:05:00,2018-07-15 19:10:00,ACC,True,not_associated,9212668.0,2018-07-03,2018-07-15
1119,b1f82cfc-3a5a-4642-b1dd-a0962b18a6c5,60740,CREW - T1 - NMCIF - MT. TAYLOR IHC,"Crew, Type 1",277015,ARENA CANYON,NM-N1S-000255,Fire - Wildfire,36.919999,-107.688055,...,2018-05-28 16:35:00,MST,2018-05-28 10:00:00,2018-05-28 15:35:00,SWC,True,not_associated,9229120.0,2018-05-26,2018-05-28
1154,62ffc800-50de-47ef-af01-edf050955801,60983,CREW - T1 - Carson Hotshots IHC,"Crew, Type 1",277015,ARENA CANYON,NM-N1S-000255,Fire - Wildfire,36.919999,-107.688055,...,2018-05-28 13:00:00,MST,2018-05-28 06:30:00,2018-05-28 12:00:00,SWC,True,not_associated,9229120.0,2018-05-26,2018-05-28
1024,be96b2e9-5e4a-41ba-8e43-be617632fc7b,58214,CREW - T1 - Sierra Hotshots IHC,"Crew, Type 1",275311,ARNOLD,CA-SNF-000261,Fire - Wildfire,37.417221,-119.361944,...,2018-04-25 14:30:00,PST,2018-04-25 13:30:00,2018-04-25 14:30:00,OSCC,True,not_associated,NaN,2018-04-24,2018-04-25
1725,7355850b-0c99-4e38-b15b-605801e5796b,2138790,CREW - T2IA - Klamath Crew 14,"Crew, Type 2 IA",276670,ASH,CA-KNF-003279,Fire - Wildfire,41.839444,-122.634444,...,2018-05-21 14:10:24,PST,2018-05-21 14:10:24,2018-05-21 14:10:24,ONCC,False,not_associated,NaN,2018-05-19,2018-05-21
1106,49605542-5122-49b9-9a21-273a0b07efe6,59047,CREW - T1 - ARROWHEAD IHC,"Crew, Type 1",277364,ASH MOUNTAIN RX,CA-KNP-000014,Fire - Prescribed,36.491944,-118.827777,...,2018-06-03 15:44:11,PST,2018-06-03 15:44:11,2018-06-03 15:44:11,OSCC,True,not_associated,NaN,2018-06-03,2018-06-03
